# Практическое занятие №2 - локализация объектов на изображениях

1. Разметка данных с помощью виджета QSL для Jupyter (https://github.com/faustomorales/qsl)
2. Расширение выборки за счет модификации размеченных данных (процедура аугоментация).
3. Знакомство с архитектурой нейронных сетей для локализации объектов на изображении.
4. Обучение нейронных сетей для детекции объектов.
5. Проверка обученной модели для обработки видео потока.
6. Трекинг объектов.
7. Распознавание сцен с помощью простого классификатора.
9. Разработка простой системы управления по данным с видео-камеры.

schema1.svg

In [ ]:
#@title
%%html
<video controls width="250">
    <source src="https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/video_1.mp4" type="video/mp4"/>
</video>
<video controls width="250">
    <source src="https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/video_3.mp4" type="video/mp4"/>
</video>
<video controls width="250">
    <source src="https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_3/video_4.mp4" type="video/mp4"/>
</video>
<video controls width="250">
    <source src="https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/video_4.mp4" type="video/mp4"/>
</video>

## Установка дополнительных пакетов для работы с изображениями

In [ ]:
!pip install -U scikit-image==0.19.3

In [ ]:
!pip install -U albumentations==1.3.0

In [ ]:
!pip install -U -q qsl

# Разметка кадров для обучающей выборки

In [ ]:
# Активация расширенных виджетов в Colab (можно закомментировать при запуске в Jupyter)
import google.colab
google.colab.output.enable_custom_widget_manager()

In [ ]:
from pandas.core.arrays import boolean
from dataclasses import dataclass
import requests
import numpy
import cv2
from collections import namedtuple
import typing
from skimage.filters import butterworth
import itertools
import albumentations as aug
import json
import pandas

import plotly.express as plte
import plotly
import plotly.graph_objects as go

import qsl

In [ ]:
%%writefile images.csv
target
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_002.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_003.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_005.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_006.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_007.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_008.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_009.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_010.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_3/image_011.jpg

In [ ]:
image_file = pandas.read_csv('images.csv')
image_file.to_dict(orient='records')

In [ ]:
labeler = qsl.MediaLabeler(
    items=image_file.to_dict(orient='records'),
    config={
        "regions": [
            {"name": "Object", "multiple": True, "options": [{"name": "bottle"}, {"name": "bag"}]}
        ]
    })
labeler.mode = "dark"
labeler

In [ ]:
with open('result.json', 'w') as file:
    file.write(json.dumps(labeler.items, sort_keys=True, indent=4))

In [ ]:
labels = None
with open('result.json', 'r') as file:
    labels = file.read()
    print(labels)

In [ ]:
!wget -O object_masks.json 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/object_masks.json'

In [ ]:
labels = None
with open('object_masks.json', 'r') as file:
    labels = file.read()
label_data = json.loads(labels)

In [ ]:
labeler = qsl.MediaLabeler(
    items=label_data,
    config={
        "regions": [
            {"name": "Type", "multiple": True, "options": [{"name": "bottle"},
                                                           {"name": "bag"}]}
        ]
    })
labeler.labels
labeler.mode = "dark"
labeler

# Нарезка изображений по объектам

In [ ]:
class Struct(object):

    def __init__(self, data):
        for name, value in data.items():
            setattr(self, name, self._wrap(value))

    def _wrap(self, value):
        if isinstance(value, (tuple, list, set, frozenset)): 
            return type(value)([self._wrap(v) for v in value])
        else:
            return Struct(value) if isinstance(value, dict) else value

    def __str__(self):
        result = '{'
        first = True
        for field in self.__dict__.keys():
            if not first:
                result += ', '
            else:
                first = False
            result += f'{field}: {str(self.__dict__[field])}'
        return result + '}'

    def __repr__(self):
        return str(self)

In [ ]:
labels = [Struct(label) for label in label_data]

In [ ]:
RawObject = namedtuple('RawObject', ['image_url', 'mask_polygon', 'label'])
Point = namedtuple('Point', ['x', 'y'])


@dataclass
class Box:
    cx: float
    cy: float
    w: float
    h: float

    def __add__(self, p: Point):
        return Box(cx=self.cx + p.x, cy=self.cy + p.y, w=self.w, h=self.h)

    @property
    def x1(self):
        return self.cx - self.w/2

    @property
    def x2(self):
        return self.cx + self.w/2

    @property
    def y1(self):
        return self.cy - self.h/2

    @property
    def y2(self):
        return self.cy + self.h/2


def make_default_background(shape):
    back = numpy.random.normal(loc=128, scale=100, size=shape).astype(numpy.uint8)
    back = butterworth(back, 0.04, False, 4, channel_axis=-1)
    return back


class MaskedObjectImage:

    def __init__(self, image, mask, label):
        self.orig_image = image
        self.mask = mask
        self.label = label
        self.__masked = None

    @property
    def masked(self):
        background = make_default_background(self.orig_image.shape)
        masked = self.orig_image.copy()
        index = self.mask == 0
        masked[index] = background[index]
        return masked

    @property
    def image(self):
        return self.orig_image

    def place(self, background: numpy.ndarray,
              pos: typing.Tuple = (0, 0), copy: boolean = True):
        result = None
        if copy:
            result = background.copy()
        else:
            result = background

        if pos[0] > background.shape[1] or pos[1] > background.shape[0]:
            raise Exception(
                'Invalid position={} for background.shape={} (out of range)'
                .format(pos, background.shape))

        index = self.mask == 0
        start_x = pos[0]
        start_y = pos[1]
        result_end_x = None
        result_end_y = None
        image_end_x = None
        image_end_y = None

        if start_x + self.mask.shape[1] <= result.shape[1]:
            result_end_x = start_x + self.mask.shape[1]
            image_end_x = self.mask.shape[1]
        else:
            result_end_x = result.shape[1]
            image_end_x = result.shape[1] - start_x

        if start_y + self.mask.shape[0] <= result.shape[0]:
            result_end_y = start_y + self.mask.shape[0]
            image_end_y = self.mask.shape[0]
        else:
            result_end_y = result.shape[0]
            image_end_y = result.shape[0] - start_y

        copy = result[start_y:result_end_y, start_x:result_end_x, :].copy()
        index = self.mask > 0
        index = index[:image_end_y, :image_end_x]
        copy[index] = self.orig_image[:image_end_y, :image_end_x, :][index]
        result[start_y:result_end_y, start_x:result_end_x, :] = copy

        return result

    @property
    def box(self):
        index = numpy.where(
            self.mask > 0
        )
        min_x = numpy.min(index[1])
        max_x = numpy.max(index[1])
        min_y = numpy.min(index[0])
        max_y = numpy.max(index[0])
        w = max_x - min_x
        h = max_y - min_y
        cx = min_x + w/2
        cy = min_y + h/2

        return Box(
            cx=cx,
            cy=cy,
            w=w,
            h=h)


class MaskedObjectsDataset:

    def __init__(self, metadata: typing.List[Struct]):
        self.metadata = metadata
        self.objects_meta = [
            RawObject(image_url=image.target,
                      label=obj.labels.Type[0],
                      mask_polygon=pandas.DataFrame.from_records(
                          [vars(point) for point in obj.points]))
            for image in self.metadata
            for obj in image.labels.polygons
            ]
        self.basic_images = {}
        self.objects = {}
        s = sorted(set(itertools.chain.from_iterable(
            [item.labels.Type for meta in metadata 
             for item in meta.labels.polygons])))
        self.label2class = {s[i]: i + 1 for i in range(0, len(s))}
        self.class2label = {i + 1: s[i] for i in range(0, len(s))}
        self.label2class['background'] = 0
        self.class2label[0] = 'background'

    def __len__(self) -> int:
        return len(self.objects_meta)

    def __getitem__(self, value) -> MaskedObjectImage:
        if not isinstance(value, slice):
            value = slice(value, value + 1)

        start = value.start
        if start is None:
            start = 0
        stop = value.stop
        if stop is None:
            stop = len(self)
        step = value.step
        if step is None:
            step = 1

        result = []
        for i in range(start, stop, step):
            result.append(self.__get_single_item(i))

        if len(result) == 1:
            result = result[0]

        return result
        
    def __get_single_item(self, i):
        if i not in self.objects:
            self.__create_object(i)

        return self.objects[i]

    def __create_object(self, i):
        object_meta = self.objects_meta[i]
        image = None
        if not object_meta.image_url in self.basic_images:
            responce = requests.get(object_meta.image_url)
            if not responce.status_code == 200:
                raise Exception("Error, cant get image {}: {}".format(
                    object_meta.image_url, responce.status_code))
            raw_bytes = numpy.frombuffer(responce.content, dtype=numpy.uint8)
            self.basic_images[object_meta.image_url] = cv2.cvtColor(
                cv2.imdecode(raw_bytes, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)

        basic_image = self.basic_images[object_meta.image_url]
        min_x = int(object_meta.mask_polygon['x'].min()*basic_image.shape[1])
        min_y = int(object_meta.mask_polygon['y'].min()*basic_image.shape[0])
        max_x = int(object_meta.mask_polygon['x'].max()*basic_image.shape[1])
        max_y = int(object_meta.mask_polygon['y'].max()*basic_image.shape[0])
        image = basic_image[min_y:max_y, min_x:max_x, :]
        mask = numpy.zeros(image.shape, dtype=numpy.uint8)
        object_meta.mask_polygon['nx'] = (
            object_meta.mask_polygon['x']*basic_image.shape[1]).map(int) - min_x
        object_meta.mask_polygon['ny'] = (
            object_meta.mask_polygon['y']*basic_image.shape[0]).map(int) - min_y
        cv2.fillPoly(mask, numpy.expand_dims(
            object_meta.mask_polygon[['nx', 'ny']].to_numpy(), axis=0),
            (255, 255, 255))
        self.objects[i] = MaskedObjectImage(
            image=image,
            mask=mask,
            label=self.label2class[object_meta.label])


In [ ]:
# %debug
dataset = MaskedObjectsDataset(metadata=labels)
len(dataset)

In [ ]:
from plotly.subplots import make_subplots


def plot_masked_objects(
    images: typing.Union[MaskedObjectImage, typing.List[MaskedObjectImage]],
    labels: typing.Dict[int, str]) -> None:
    if isinstance(images, MaskedObjectImage):
        images = [images]
    for img in images:
        fig = make_subplots(
            horizontal_spacing=0.0, vertical_spacing=0.0,
            rows=1, cols=3, shared_yaxes=True, shared_xaxes=True,
            subplot_titles=[f"{labels[img.label]}[{str(img.label)}]"])
        fig.add_trace(go.Image(z=img.orig_image), row=1, col=1)
        fig.add_trace(go.Image(z=img.mask), row=1, col=2)
        fig.add_trace(go.Image(z=img.masked), row=1, col=3)
        fig.show()

In [ ]:
plot_masked_objects(dataset[0], dataset.class2label)

# Расширение обучающей выборки 

На практике не всегда удается собрать достаточно размеченного материала для качественного обучения нейронной сети. Кроме того, использование большого массива изображений без должного анализа состава приводит к разбалансировке процесса обучения. Например, в общем потоке изображений какие-либо классы объектов могут быть представлены незначительным числом примеров и по этой причине их распознавание во время обучения будет иметь малую значимость на фоне остальных классов. В этой связи все современные модели при обучении предполагают в определенной степени крапотливое конструирование синтетического набора данных на основе исходных изображений. Подробный обзор стратегий расширения выборки за счет изменения исходных изображений представлен в статье "A Comprehensive Survey of Image Augmentation Techniques for Deep Learning" (https://arxiv.org/abs/2205.01491).

Далее будем использовать следующий способ расширения выборки изображений.
1. Извлечение изображений интересующих нас объектов из исходных данных.
2. Создание набора фоновых изображений для размещения объектов.
3. Определение набора трасформаций изображений объектов и фона, включая масштабирование, отражения, повороты, обрезку, изменение цвета/насыщенности, наложение шума.
4. Размещение объектов на фоне в случайных местах.

В настоящее время нет необходимости реализовывать с нуля все шаги синтеза изображений, т.к. существуюет большой набор готовых решений. Наиболее популярными решениями служат следующие пакеты Python.

1. https://pytorch.org/vision/stable/transforms.html
2. https://www.tensorflow.org/tutorials/images/data_augmentation?hl=en
3. https://augmentor.readthedocs.io/en/master/
4. https://github.com/albumentations-team/albumentations

Важным вопросом при расширении выборки является выбор параметров. Он может быть выполнен как эмпирически, так и путем оптимизации. В последнем случае используют либо целевую модель, либо её малый эквивалент с целью вариации параметров и оценки качества обучения. Логичным итогом такого рода работы становятся так называемые "системы автоаугоментации", в которые встраивается универсальная внутренняя модель и алгоритм оптимизации для подбора параметров трансформаций. Они могут использоваться для расширения новых наборов данных без дополнительных усилий со стороны разработчика (но ценой значительного расхода вычислений, т.к. необходимо многократно проводить процесс обучения моделей). Более подробно об основной идее такого автоматического подхода можно почитать в статье сотрудников компании Google (https://arxiv.org/abs/1805.09501), а также изучить похожую реализацию в пакете deepaugment - https://github.com/barisozmen/deepaugment (к сожалению, пакет больше не поддерживается, последнее обновление исходного кода было в 2019 году).
Далее будем использовать более простой "эмпирический способ".



In [ ]:
transform = aug.Compose([
    aug.CropAndPad(percent=(0.2, 0.2), keep_size=False,
                   pad_mode=cv2.BORDER_CONSTANT),
    aug.LongestMaxSize(max_size=300),
    aug.Rotate(p=0.9, border_mode=cv2.BORDER_CONSTANT,
               value=(0, 0, 0), mask_value=(0, 0, 0)),
    # aug.RandomCrop(width=50, height=50, p=0.25),
    aug.HorizontalFlip(p=0.5),
    aug.VerticalFlip(p=0.5),
    aug.RandomScale(scale_limit=0.5, always_apply=True),
    aug.MotionBlur(),
    aug.RGBShift(always_apply=True,
                 r_shift_limit=32, g_shift_limit=32, b_shift_limit=32, p=0.9),
    aug.ChannelShuffle(p=0.3), # new
    aug.HueSaturationValue(always_apply=True, p=1.0),
    aug.RandomBrightnessContrast(p=0.2),
    aug.GaussNoise(always_apply=True)
])

In [ ]:
import random

def sample_image(
    images: typing.List[MaskedObjectImage],
    transf: aug.Compose) -> MaskedObjectImage:
    choice = int(random.random()*len(images))
    sample = transf(image=images[choice].image, mask=images[choice].mask)
    return MaskedObjectImage(image=sample['image'],
                             mask=sample['mask'],
                             label=images[choice].label)

In [ ]:
num_of_samples = 1
plot_masked_objects([
    sample_image(images=[dataset[1:2]], transf=transform)
    for i in range(0, num_of_samples)], dataset.class2label)

In [ ]:
def generate_objects_image(
    objects: MaskedObjectsDataset,
    size: typing.Tuple[int, int, int],
    N: int,
    transformation: aug.Compose,
    background: numpy.ndarray = None
    ) -> typing.Tuple[numpy.ndarray, typing.List[Box], typing.List[int]]:

    if background is None:
        background = butterworth(make_default_background(size), 0.1, False, 4,
                             channel_axis=-1)
    result = background
    boxes = []
    labels = []
    for i in range(0, N):
        image = sample_image(images=objects, transf=transformation)
        pos = Point(
            x=int(random.random()*(result.shape[1] - image.mask.shape[1])),
            y=int(random.random()*(result.shape[0] - image.mask.shape[0])))
        result = image.place(
            background=result,
            pos=pos
        )
        boxes.append(image.box + pos)
        labels.append(image.label)
                
    return result, boxes, labels


def plot_image_with_objects(
    image: MaskedObjectImage,
    boxes: Box,
    labels: typing.List[int],
    labels_map: typing.Dict[int, str]) -> None:
    f = plte.imshow(image)
    for i in range(0, len(boxes)):
        b = boxes[i]
        label = labels_map[labels[i]]
        f.add_trace(go.Scatter(
            x=[b.x1, b.x1, b.x2, b.x2, b.x1],
            y=[b.y1, b.y2, b.y2, b.y1, b.y1],
            name=label))
        f.add_annotation(x=b.x1, y=b.y1,
                text=label,
                showarrow=False,
                yshift=10)
    f.show()

In [ ]:
image, image_boxes, image_labels = generate_objects_image(
    objects=dataset,
    size=(768, 1280, 3),
    N=10,
    transformation=transform
)
plot_image_with_objects(
    image=image,
    boxes=image_boxes,
    labels=image_labels,
    labels_map=dataset.class2label
)

# Подготовка выборки для валидации моделей
Как и в первом практическом занятии необходим набор примеров для проверки качества обучаемой модели, а также определения оптимальных параметров процесса настройки весов для исключения переобучения.

In [ ]:
%%writefile images-validation.csv
target
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/image012.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/image013.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/image014.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/image015.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/image016.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/image017.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/image018.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/image019.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/image020.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/image021.jpg

In [ ]:
import pandas

image_validation_file = pandas.read_csv('images-validation.csv')
image_validation_file

In [ ]:
import qsl

labeler = qsl.MediaLabeler(
    items=image_validation_file.to_dict(orient='records'),
    config={
        "regions": [
            {"name": "Object", "multiple": True, "options": [{"name": "bottle"},
                                                             {"name": "bag"}]}
        ]
    })
labeler.mode = "dark"
labeler

In [ ]:
import json
with open('result-validation.json', 'w') as file:
    file.write(json.dumps(labeler.items, sort_keys=True, indent=4))

In [ ]:
!wget -O object-validation.json 'https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/validation.json'

In [ ]:
labels = None
with open('object-validation.json', 'r') as file:
    labels = file.read()
validation_labels = json.loads(labels)

In [ ]:
labeler = qsl.MediaLabeler(
    items=validation_labels,
    config={
        "regions": [
            {"name": "Object", "multiple": True, "options": [{"name": "bottle"},
                                                             {"name": "bag"}]}
        ]
    })
labeler.mode = "dark"
labeler

# Использование предобученной модели Fastern R-CNN для детекции

Классическая задача детекции состоит в поиске объектов на изображении. Основные идеи детекции с помощью нейронных сетей группируются в семейства моделей. Одно из таких семейств использует идею генерации кандидатов детектируемых объектов по карте признаков, расчитываемой с помощью сети для задач классификации. Результатирующее положение и принадлежность объекта классу вычисляется с помощью полносвязной сети на основе списка кандидатов и карты признаков.

Ключевой идеей является генерация подходящего набора канидатов по исходному изображению (RoI-region of interest). Задача осложняется многообразием объектов, которые необходимо находить на изображении, различных положений и соотношения сторон, а также переменным числом сущностей на изображении. 

Первой идеей таких моделей является использование фиксированного набора кандидатов с такими признаком как "отсутствие объекта". На выход всегда выдается одинаковое число объектов, но за счет фильтрации по признаку "отсутствие объекта" получается переменное число, которое не может превышать  максимально заложенное в модели.

Второй идеей является использование фиксированного набора "опорных областей интереса" на изображении для детектируемых объектов с различным масштабированием и отношением сторон (Region Proposal - RP). Они задают дополнительную размерность в карте признаков и позволяют эффективнее вычислять итоговое положение и размер объектов по сравнению с прямым расчетом. Модель всего лишь корректирует параметры опорных областей, что гораздо эффективнее расчета абсолютных координат и размеров объектов на изображении. Таким образом, в карте признаков на каждый "пиксель" приходится $k$ кандидатов областей, для каждой из которых задается вероятность нахождения объекта и его класс (в том чсиле отсутствие какого-либо объекта). На практике использование опорных областей в изображении оказывается универсальнее и эффективнее с точки зрения числа параметров и вычислительной сложности по сравнению с предварительной генерацией достаточного числа произвольных прямоугольников и их классификации на предмет наличия объектов внутри.

Третья идея состоит в переиспользовании результатов наиболее трудоемких вычислений различными подсистемами модели. Речь идёт о вычислении карты признаков исходного изображения на основе классических архитектур нейросетей для классификации изображений с отключенным финальным полносвязным слоем.

https://arxiv.org/pdf/1908.03673.pdf
https://paperswithcode.com/sota/object-detection-on-coco-2017-val

| Модель | Год | Тип | Ключевые идеи | Особенности |
| :- | :--: | :--: | :- | :- |
| R-CNN | | двухшаговая на основе <br> областей интереса |  Selective Search для RoI + CNN для создания <br> feature map в каждом RoI + SVM для классификации | Медленная работа, фиксированный алгоритм генерации RoI|
| Fast R-CNN |  | двухшаговая на основе <br> областей интереса | CNN (feature map) расчитывается однократно, <br> далее нарезается по итогам работы Selective Search на <br> Feature Map для каждого RoI, затем слой RoI Pooling с окном <br> фиксированного размера и заключительные две FC-сети для <br> классификации и коррекции RoI в ограничивающий <br> прямоугольник объекта| ~ в 25 раз быстрее R-CNN, но проблема с Selective Search осталась|
| Fastern R-CNN | [2015](https://arxiv.org/abs/1506.01497) | двухшаговая на основе <br> областей интереса |Замена Selective Search на Region Proposal Network: <br> вместо эвристического алгоритма используется вложенная <br> нейронная сеть (RPN), которая вычисляет коррекцию <br> фиксированного набора опорных прямоугольников| ~ в 10 раз быстрее Fast R-CNN (~5-10 FPS)|
|SSD|[2015](https://arxiv.org/abs/1512.02325)|одношаговая (по сетке?)||
|RetinaNet | [2017](https://arxiv.org/abs/1708.02002) | одношаговая (по сетке?) | Точнее Yolo, быстрее Faster R-CNN, но проигрывает ей в точности|
| Yolo | [2015-2023](https://arxiv.org/pdf/2301.05586v1.pdf) | одношаговая по сетке | You Only Look Once - разбивка изображения на сетку, <br> использвоание комбинации различных техник для создания | Наиболее быстрая модель ~ в 10 раз быстрее Faster-R-CNN <br> (~50-150 FPS на GPU), однако могут быть проблемы в детекции объектов <br> разных масштабов |
| FCos | [2019](https://arxiv.org/abs/1904.01355) | одношаговая (?) без сетки||








In [ ]:
import numpy
from torchvision.io.image import read_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
import torch
from PIL import Image


!lscpu
print(f'GPU count = {torch.cuda.device_count() if torch.cuda.is_available() else "0"}')

In [ ]:
!nvidia-smi

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

In [ ]:
img = torch.from_numpy(
    numpy.moveaxis(image.astype(numpy.uint8), 2, 0))

# Step 1: Initialize model with the best available weights
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9).to(device)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms().to(device)

In [ ]:
# Step 3: Apply inference preprocessing transforms
batch = [preprocess(img).to(device)]

In [ ]:
# Step 4: Use the model and visualize the prediction
result = model(batch)

In [ ]:
# %debug
box = img
for res in result:
    prediction = res
    pred_labels = [weights.meta["categories"][i] for i in prediction["labels"].tolist()]
    box = draw_bounding_boxes(box, boxes=prediction["boxes"],
                              labels=pred_labels,
                              colors="red",
                              width=4, font_size=30)

fig = plte.imshow(to_pil_image(box.detach()))
fig.show()

In [ ]:
result

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary

summary(model, input_data=[batch])

In [ ]:
InputOutput = namedtuple('InputOutput', ['input', 'output'])

def get_activation(name, trace):
    def hook(model, input, output):
        trace[name] = InputOutput(input[0].detach(), output.detach())

    return hook

In [ ]:
trace = {}
model.roi_heads.box_head[5].register_forward_hook(get_activation('head-5', trace))
model.roi_heads.box_head[6].register_forward_hook(get_activation('head-6', trace))
model.roi_heads.box_predictor.cls_score.register_forward_hook(get_activation('cls', trace))
model.roi_heads.box_predictor.bbox_pred.register_forward_hook(get_activation('bbox', trace))
model.rpn.head.cls_logits.register_forward_hook(get_activation('cls_log_prop', trace))
model.rpn.head.bbox_pred.register_forward_hook(get_activation('bbox_prop', trace))
output = model(batch)
output

In [ ]:
for key, value in trace.items():
    print(f"{key}: {value.input.shape} => {value.output.shape}")

In [ ]:
import torch.nn.functional as F

threshold = 0.75
cls = trace['cls'].output[:, 1:]
cls_prob_all = F.softmax(cls, -1)
cls_prob_thr = (cls_prob_all > threshold).float()
cls_prob_x = cls_prob_all * cls_prob_thr

obji = torch.argmax(cls_prob_x, 1)
bboxi = cls_prob_all[range(cls_prob_all.shape[0]), obji]
bboxes = trace['bbox'].output[bboxi > threshold]
clses = obji[bboxi > threshold]
conf = cls_prob_all[range(cls_prob_all.shape[0]), obji][bboxi > threshold]

In [ ]:
[{
    'class': i + 1, 'label': weights.meta["categories"][i + 1],
    'count': len(conf[clses == i]),
    'prob': f'{int(min(conf[clses == i]).item()*100)}-{int(max(conf[clses == i]).item()*100)}%'
 } for i in clses.unique().tolist()]

TODO: 
1. Понять почему так много объектов, возможно кого-то нужно отсекать по размеру bbox? (разобраться по исходникам класса).
2. Разобраться как вычисляются координаты bbox из bbox-регресии для того чтобы реконструировать вычисление выхода сетки из промежуточных результатов trace['bbox'] и обновить таблицу выше.

# Изменение числа классов в Faster R-CNN

In [ ]:
import copy

tunned_model = copy.deepcopy(model)

In [ ]:
for param in tunned_model.parameters():
    param.requires_grad = False

In [ ]:
print(model.roi_heads.box_predictor.cls_score)
print(model.roi_heads.box_predictor.bbox_pred)

In [ ]:
import torchvision


num_of_classes = 2
tunned_model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
    in_channels=1024, num_classes=num_of_classes + 1)
summary(tunned_model, input_data=[batch])

# Подготовка датасета для дообучения модели

In [ ]:
from collections import namedtuple
import os
import shutil


image_feeder = lambda: generate_objects_image(
    objects=dataset,
    size=(768, 1280, 3),
    N=4,
    transformation=transform
)

output_folder = './output'
if os.path.isdir(output_folder):
    shutil.rmtree(output_folder)
os.mkdir(output_folder)

N = 200
part = 10
learn_dataset = []
for i in range(0, N):
    image, boxes, image_labels = image_feeder()
    image_file = f'img{i}.png'
    image_path = f'{output_folder}/{image_file}'
    image_data = Image.fromarray(image.astype(numpy.uint8))
    image_data.save(image_path)
    for k in range(0, len(boxes)):
        learn_dataset.append({
            'image': image_file,
            'cx': boxes[k].cx/image.shape[1],
            'cy': boxes[k].cy/image.shape[0],
            'w': boxes[k].w/image.shape[1],
            'h': boxes[k].h/image.shape[0],
            'label': image_labels[k]
        })
    if i % part == 0:
        print(f'{int(i/N*100)}% ({i}/{N}) were generated')
pandas.DataFrame.from_records(learn_dataset).to_csv(
    f'{output_folder}/labels.txt', index=False)
shutil.make_archive('dataset', 'zip', root_dir=output_folder)
print('Done')

см. https://discuss.pytorch.org/t/how-can-l-load-my-best-model-as-a-feature-extractor-evaluator/17254/6

In [ ]:
# output = []
# def hook()
# model.roi_heads.Conv2dNormActivation.ReLU.register_module_forward_hook(hook)

In [ ]:
import os
import typing
import torch
import torchvision
import numpy
import pandas
import matplotlib.pyplot as plt
from collections import namedtuple


LabeledImageSample = namedtuple('LabeledImageSample', ['image', 'labels'])
xf = ['cx', 'w']
yf = ['cy', 'h']


class Dataset(torch.utils.data.Dataset):

    def __init__(self, folder: str, device: torch.device,
                 scale_bbox=False, num=None) -> None:
        self.__folder = folder
        self.__image = {}
        self.__bbox = {}
        self.__device = device
        path = os.path.join(folder, 'labels.txt')
        self.meta = pandas.read_csv(path, sep=',')
        self.image_files = self.meta['image'].unique()
        self.__num = num

    def __len__(self) -> int:
        if self.__num is not None:
            return self.__num

        return len(self.image_files)

    def __getitem__(self, item: int) -> pandas.DataFrame:
        file = self.image_files[item]
        index = self.meta['image'] == file
        if file not in self.__image:
            self.__image[file] = torchvision.io.read_image(os.path.join(
                self.__folder, file))
            self.meta.loc[index, xf] = self.meta.loc[
                index, xf]*self.__image[file].shape[2]
            self.meta.loc[index, yf] = self.meta.loc[
                index, yf]*self.__image[file].shape[1]
            self.meta['p1x'] = self.meta['cx'] - self.meta['w']/2
            self.meta['p2x'] = self.meta['cx'] + self.meta['w']/2
            self.meta['p1y'] = self.meta['cy'] - self.meta['h']/2
            self.meta['p2y'] = self.meta['cy'] + self.meta['h']/2

        return LabeledImageSample(image=self.__image[file],
                                  labels=self.meta.loc[index])

In [ ]:
data = Dataset(folder="output", device=device, num=None)

In [ ]:
def show_image(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = torchvision.transforms.functional.to_pil_image(img)
        axs[0, i].imshow(numpy.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

In [ ]:
show_image(data[33].image)

Подготовка датасета для валидации обучения.

In [ ]:
validation_labels[0]['labels']['boxes'][0]['labels']['Object'][0]

In [ ]:
import os
import shutil

output_folder = 'validation'
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder)

val_labels = []

for item in validation_labels:
    responce = requests.get(item['target'])
    if not responce.status_code == 200:
        raise Exception("Error, cant get image {}: {}".format(
            item['target'], responce.status_code))
    fname = item['target'][(item['target'].rfind('/')+1):]
    path = os.path.join(output_folder, fname)
    with open(path, 'wb') as file:
        file.write(responce.content)
    for box in item['labels']['boxes']:
        p1 = box['pt1']
        p2 = box['pt2']
        w = p2['x'] - p1['x']
        h = p2['y'] - p1['y']
        val_labels.append({
            'image': fname,
            'cx': p1['x'] + w/2,
            'cy': p1['y'] + h/2,
            'w': w,
            'h': h,
            'label': dataset.label2class[box['labels']['Object'][0]]
        })
pandas.DataFrame.from_records(val_labels).to_csv(
    os.path.join(output_folder, 'labels.txt'))

In [ ]:
validation_data = Dataset(folder="validation", device=device, num=None)

In [ ]:
show_image(validation_data[0].image)

# Дообучение модели Fastern RCNN для детекции новых классов

https://github.com/johschmidt42/PyTorch-Object-Detection-Faster-RCNN-Tutorial

In [ ]:
import time
import math


losses = ['loss_classifier', 'loss_box_reg']


class Model:

    def __init__(self, model, classes, device, preprocess, labels,
                 scheduler=None, lr=0.1, modify=True) -> None:
        self.__device = device
        self.__orig_model = model
        self.__tunned_model = copy.deepcopy(self.__orig_model)
        if modify is True:
            self.__modify(classes=classes)
        self.__tunned_model.to(self.__device)
        self.__preprocess = preprocess
        self.__critery = torchvision.ops.complete_box_iou_loss
        self.__optimizer = torch.optim.SGD(
            self.__tunned_model.parameters(), lr=lr, momentum=0.9)
        if scheduler is None:
            scheduler = torch.optim.lr_scheduler.StepLR(
                self.__optimizer, step_size=3, gamma=0.5,verbose=True)
        self.__scheduler = scheduler
        self.__labels = labels

    def __modify(self, classes: int, retrain_all: bool = False):
        model = self.__tunned_model
        if retrain_all is False:
            for param in model.parameters():
                param.requires_grad = False
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        output = torchvision.models.detection.faster_rcnn
        model.roi_heads.box_predictor = output.FastRCNNPredictor(
            in_features, classes)

    def get_model(self):
        return self.__tunned_model

    def __prepare_inputs(self,
                         item: LabeledImageSample):
        inputs = item.image.float().to(self.__device)/255
        labels = {
            'boxes': torch.from_numpy(
                item.labels[['p1x', 'p1y', 'p2x', 'p2y']].to_numpy()
                ).to(self.__device),
            'labels': torch.from_numpy(
                item.labels['label'].to_numpy()).to(self.__device)
        }
        if len(inputs.shape) < 4:
            inputs = inputs.reshape(-1, 
                                    inputs.shape[0],
                                    inputs.shape[1],
                                    inputs.shape[2])
            labels = [labels]

        return inputs, labels

    def train(self,
              train_data: torch.utils.data.DataLoader,
              epoch_count: int,
              print_info: bool = True,
              validation_data: torch.utils.data.DataLoader = None):
        since = time.time()
        self.__tunned_model.train()
        loss_log = []
        for i in range(epoch_count):
            if print_info:
                print(f"Epoch {i+1} / {epoch_count}")
            result_loss = {}
            validation_loss = {}
            for key in losses:
                result_loss[key] = 0
                validation_loss[key] = 0

            train_count = 0
            for item in train_data:
                train_count = train_count + 1
                inputs, labels = self.__prepare_inputs(item)
                self.__optimizer.zero_grad()
                with torch.set_grad_enabled(True):
                    loss = self.__tunned_model(inputs, targets=labels)
                    for key in losses:
                        result_loss[key] = result_loss[key] + loss[key].detach().cpu()
                    loss_bw = loss['loss_classifier'] + loss['loss_box_reg']
                    loss_bw.backward()
                    self.__optimizer.step()

            if validation_data is not None:
                validation_count = 0
                for item in validation_data:
                    validation_count = validation_count + 1
                    inputs, labels = self.__prepare_inputs(item)
                    with torch.set_grad_enabled(False):
                        loss = self.__tunned_model(inputs, targets=labels)
                        for key in losses:
                            validation_loss[key] = validation_loss[key] + loss[key].detach().cpu()

            for key in losses:
                result_loss[key] = result_loss[key]/train_count
                print(f"    [{i+1}/{epoch_count}][train][{key}] " +
                      f"loss = {result_loss[key]}")
                if validation_data is not None:
                    validation_loss[key] = validation_loss[key]/validation_count
                    print(f"    [{i+1}/{epoch_count}][validation][{key}] " +
                          f"loss = {validation_loss[key]}")
            loss_log.append({'train': result_loss, 'validation': validation_loss})

            self.__scheduler.step()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

        return loss_log
                    
            

    def predict(self, inputs: torch.Tensor, measure_inference_time: bool = False):
        if measure_inference_time:
            since = time.time()
        self.__tunned_model.eval()
        result = self.__tunned_model(inputs)
        labels = [self.__labels[i.detach().cpu().item()] for i in result[0]['labels']]

        if measure_inference_time:
            time_elapsed = time.time() - since
            print(f'Inference has been completed in {time_elapsed // 60:.0f}m' +
                  f' {time_elapsed % 60:.0f}s {int((time_elapsed - math.floor(time_elapsed))*1000)}ms')
        return result, labels

In [ ]:
class_names = dataset.class2label
new_model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.5).to(device)
tunned_model = Model(model=new_model, classes=3, device=device, preprocess=preprocess,
                     labels=class_names, lr=0.025)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
!nvidia-smi

In [ ]:
def draw_with_box_in_image(
    image: numpy.ndarray,
    labels:  typing.Union[typing.List[str], typing.Mapping[int, str]] = None,
    meta: typing.Mapping[int, str] = None) -> None:
    box = image

    if labels is not None:
        if meta is not None:
            pred_labels = [meta[i] for i in labels["labels"].tolist()]
        else:
            pred_labels = [str(i) for i in labels["labels"]]
        box = draw_bounding_boxes(box, boxes=torch.Tensor(labels["boxes"]),
                                  labels=pred_labels,
                                  colors="red",
                                  width=4, font_size=30)

    return box


def draw_with_bbox(
    image: numpy.ndarray,
    labels:  typing.Union[typing.List[str], typing.Mapping[int, str]] = None,
    meta: typing.Mapping[int, str] = None) -> None:

    box = draw_with_box_in_image(image=image, labels=labels, meta=meta)

    fig = plte.imshow(to_pil_image(box.detach()))
    fig.show()


def df2labels(df: pandas.DataFrame) -> typing.Mapping[str, typing.Union[
    typing.List[numpy.array], typing.List[int]]]:
    return {'boxes': df[['p1x', 'p1y', 'p2x', 'p2y']].to_numpy(),
            'labels': df['label'].to_numpy()}


In [ ]:
data[0].image.shape

In [ ]:
# %debug
draw_with_bbox(image=data[0].image, labels=df2labels(data[0].labels),
               meta=class_names)

In [ ]:
# %debug
loss_log = tunned_model.train(
    train_data=DataLoader(data,
                          batch_size=None,
                          shuffle=True,
                          num_workers=0),
    epoch_count=15,
    validation_data=validation_data
    )

In [ ]:
def plot_learn_metrics(loss_log) -> None:
    loss = pandas.DataFrame({
        'train_box': [loss['train']['loss_box_reg'].item() for loss in loss_log],
        'train_class': [loss['train']['loss_classifier'].item() for loss in loss_log],
        'validation_box': [loss['validation']['loss_box_reg'].item() for loss in loss_log],
        'validation_class': [loss['validation']['loss_classifier'].item() for loss in loss_log]
        })
    fig = make_subplots(
                horizontal_spacing=0.1, vertical_spacing=0.1,
                rows=2, cols=2, # shared_yaxes=True,
                shared_xaxes=True,
                subplot_titles=[
                    "Ошибка локализации (обучение)",
                    "Ошибка классификации (обучение)",
                    "Ошибка локализации (валидация)",
                    "Ошибка классификации (валидация)",
                    ])
    fig.add_trace(go.Scatter(x=loss.index, y=loss['train_box']), row=1, col=1)
    fig.add_trace(go.Scatter(x=loss.index, y=loss['train_class']), row=1, col=2)
    fig.add_trace(go.Scatter(x=loss.index, y=loss['validation_box']), row=2, col=1)
    fig.add_trace(go.Scatter(x=loss.index, y=loss['validation_class']), row=2, col=2)
    fig.show()

In [ ]:
plot_learn_metrics(loss_log)

In [ ]:
# for i in range(0, len(data)):
#    pred = tunned_model.predict([(data[1].image.float()/255).to(device)])
#    if len(pred[0][0]['boxes']) > 0:
#        print(f"image {i} has {len(pred[0][0]['boxes'])} boxes")

In [ ]:
i = 2
pred = tunned_model.predict([(data[i].image.float()/255).to(device)])
draw_with_bbox(data[i].image, pred[0][0], meta=class_names)
pred

In [ ]:
tunned_model.get_model().roi_heads.score_thresh = 0.75
first = list(dataset.basic_images.keys())[2]
image = torch.Tensor(dataset.basic_images[first]).permute([2, 0, 1])/255
pred = tunned_model.predict([image.to(device)])
pred

In [ ]:
draw_with_bbox((image*255).to(torch.uint8), pred[0][0], meta=class_names)

In [ ]:
i = 1
tunned_model.get_model().roi_heads.score_thresh = 0.75
image = torch.Tensor(validation_data[i].image)/255 #.permute([2, 0, 1])/255
pred = tunned_model.predict([image.to(device)])
draw_with_bbox((image*255).to(torch.uint8), pred[0][0], meta=class_names)
pred


# Создание видео файла с результатами распознавания

In [ ]:
!pip install video-cli

In [ ]:
!wget -O video-5.mp4 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/video_4.mp4'

In [ ]:
import cv2


def process_video(input: str, model: typing.Callable[[torch.Tensor], None],
                  output: str) -> None:
    """ Function processes an input video file by a model and create an output 
        video file.
    """
    cap = cv2.VideoCapture(input)
    fps = cap.get(cv2.CAP_PROP_FPS)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = int(length / 10)
    ret, frame = cap.read()
    h, w, _ = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*"VP90")
    writer = cv2.VideoWriter(output, fourcc, fps, (w, h))

    count = 1
    while ret: 
        frame = model(frame)
        writer.write(frame)
        ret, frame = cap.read()

        if count % step == 0:
            print(f'{int(count/length*100)}% frames processed ({count}/{length})')
                        
        count += 1

    writer.release()
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
def map_box(image: numpy.ndarray,
            device: torch.device,
            model: typing.Callable[[typing.Any], typing.Tuple],
            meta: typing.Mapping[int, str] = None) -> None:
    imag = torch.Tensor(
        numpy.transpose(image, axes=[2, 0, 1]).astype(float)).to(device)/255
    pred = model.predict([imag])
    return draw_with_box_in_image(image=(imag*255).byte(),
                                  labels=pred[0][0],
                                  meta=meta).cpu().numpy().transpose(1, 2, 0)

In [ ]:
import functools

model_operator = functools.partial(map_box, device=device,
                                   model=tunned_model, meta=class_names)

In [ ]:
process_video(input='video-5.mp4',
              model=model_operator, output='video-5-out-1.webm')

In [ ]:
from IPython.display import Video

# Video('video-5-out-1.webm', width=600, embed=True)

# Устранение артефактов в обучающей выборке

1.   Новый пункт
2.   Новый пункт



In [ ]:
!wget -O object_masks_fixed_images.json 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/object_masks_fixed_images.json'

In [ ]:
fixed_labels = None
with open('object_masks_fixed_images.json', 'r') as file:
    fixed_labels = file.read()
fixed_label_data = json.loads(fixed_labels)
labeler = qsl.MediaLabeler(
    items=fixed_label_data,
    config={
        "regions": [
            {"name": "Type", "multiple": True, "options": [{"name": "bottle"},
                                                           {"name": "bag"}]}
        ]
    })
labeler.labels
labeler.mode = "dark"
labeler

In [ ]:
fixed_labels = [Struct(label) for label in fixed_label_data]
fixed_dataset = MaskedObjectsDataset(metadata=fixed_labels)
fixed_image_feeder = lambda: generate_objects_image(
    objects=fixed_dataset,
    size=(768, 1280, 3),
    N=4,
    transformation=transform
)

output_folder = './output_fixed'
if os.path.isdir(output_folder):
    shutil.rmtree(output_folder)
os.mkdir(output_folder)

N = 200
part = 10
learn_dataset = []
for i in range(0, N):
    image, boxes, image_labels = fixed_image_feeder()
    image_file = f'img{i}.png'
    image_path = f'{output_folder}/{image_file}'
    image_data = Image.fromarray(image.astype(numpy.uint8))
    image_data.save(image_path)
    for k in range(0, len(boxes)):
        learn_dataset.append({
            'image': image_file,
            'cx': boxes[k].cx/image.shape[1],
            'cy': boxes[k].cy/image.shape[0],
            'w': boxes[k].w/image.shape[1],
            'h': boxes[k].h/image.shape[0],
            'label': image_labels[k]
        })
    if i % part == 0:
        print(f'{int(i/N*100)}% ({i}/{N}) were generated')
pandas.DataFrame.from_records(learn_dataset).to_csv(
    f'{output_folder}/labels.txt', index=False)
shutil.make_archive('dataset_fixed', 'zip', root_dir=output_folder)
print('Done')

In [ ]:
fixed_marks_data = Dataset(folder="output_fixed", device=device, num=None)

In [ ]:
class_names = dataset.class2label
new_model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.5).to(device)
fixed_marks_tunned_model = Model(model=new_model, classes=3, device=device, preprocess=preprocess,
                     labels=class_names, lr=0.025)
# %debug
corrected_loss_log = fixed_marks_tunned_model.train(
    train_data=DataLoader(fixed_marks_data,
                          batch_size=None,
                          shuffle=True,
                          num_workers=0),
    epoch_count=15,
    validation_data=validation_data
    )

In [ ]:
plot_learn_metrics(corrected_loss_log)

In [ ]:
import functools

fixed_model_operator = functools.partial(map_box, device=device,
                                         model=fixed_marks_tunned_model, meta=class_names)
process_video(input='video-5.mp4',
              model=fixed_model_operator, output='video-5-out-marks-fixed.webm')

# Обучение на изображениях со специфичными фонами

In [ ]:
%%writefile images-background.csv
target
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/background/image_001.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/background/image_005.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/background/image_011.jpg

In [ ]:
image_validation_file = pandas.read_csv('images-background.csv')
image_validation_file

In [ ]:
def generate_background(backgrounds: typing.Sequence[numpy.ndarray],
                        size: typing.Tuple[int, int, int]):
    transform = aug.Compose([
        aug.CropAndPad(percent=(1.0, 1.0), keep_size=False,
                   pad_mode=cv2.BORDER_REFLECT),
        # aug.LongestMaxSize(max_size=300),
        aug.Rotate(p=1.0, border_mode=cv2.BORDER_REFLECT,
                value=(0, 0, 0), mask_value=(0, 0, 0)),
        #aug.HorizontalFlip(p=0.5),
        #aug.VerticalFlip(p=0.5),
        aug.RandomScale(scale_limit=0.5, always_apply=True),
        aug.RandomCrop(width=size[0], height=size[1], p=1.0),
        aug.MotionBlur(),
        aug.RGBShift(always_apply=True,
                    r_shift_limit=20, g_shift_limit=20, b_shift_limit=20),
        aug.HueSaturationValue(always_apply=True, p=1.0),
        aug.RandomBrightnessContrast(p=0.2),
        aug.GaussNoise(always_apply=True)
    ])